In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import concurrent
import pickle
from scipy.interpolate import griddata
from scipy import interpolate

import matplotlib
from mpl_toolkits import mplot3d
font = {'size'   : 16}
matplotlib.rc('font', **font)

In [2]:
file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\hex_minor_Ichannel.h5"
print(os.path.isfile(file_path))
out_folder = 'results'
try:
    os.stat(out_folder)
except:
    os.mkdir(out_folder) 
poles = [0, 1, 2]
poles_dict = {0:'A', 1:'B', 2:'C'}

True


In [4]:
# poles = [0]
def filter_data(data, bin_step=100):
    # to speed up, filter the high frequency stuff
    t = data[:, 0].copy()
    n = int(len(t)/bin_step)
    data_bin = np.zeros((n, data.shape[1]))
    for i in range(data.shape[1]):
        data_bin[:, i] = binned_statistic(t, data[:, i], bins=int(len(t)/bin_step), statistic='mean').statistic
    return data_bin

hyst_all = []
hyst_file_path = file_path
# get the rl coefficients
with open('rl_coeff.p', 'rb') as f:
    rl_coeff = pickle.load(f)
hp_minmax = np.zeros((3, 2))
with h5py.File(hyst_file_path, 'r') as f:
    for pole in poles:
        hyst_pole = []
        pole_grp = f['pole{}'.format(pole)]
        for step in pole_grp.keys():
            hp = pd.DataFrame(pole_grp[step+'/hallprobe/data'][:])
            hx = pd.DataFrame(pole_grp[step+'/hexapole/data'][:])
            total_t = hp[0].values[-1] - hp[0].values[0]
            bins = np.arange(hp[0].iloc[0], hp[0].iloc[-1], 1/100)

            hp['bins'] = pd.cut(hp[0], bins, right=False)
            hp = hp.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
            hp.columns = ['hpA', 'hpB', 'hpC']
            hx['bins'] = pd.cut(hx[0], bins, right=False)
            hx = hx.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
            hx.columns = ['hxA', 'hxB', 'hxC']
            

            # add to one dataframe, flip hp for convenience
            hp *= -1
            hyst_step = pd.concat([hp, hx], axis=1)
            # get the min and max values
            hpmin = np.min(hp.values)
            hpmax = np.max(hp.values)
            if hp_minmax[pole, 0] > hpmin:
                hp_minmax[pole, 0] = hpmin
            if hp_minmax[pole, 1] < hpmax:
                hp_minmax[pole, 1] = hpmax
            # remove the part where we hit compliance
#             complianceV = 20/np.sqrt(rl_coeff[pole]['R']**2 + (2*np.pi*rl_coeff[pole]['L']/total_t)**2)
            complianceV = 9
            hyst_step[hx.columns] = hyst_step[hyst_step[hx.columns].abs()<complianceV][hx.columns]
            hyst_step.dropna(inplace=True)
            
            # add to the list
            hyst_pole.append(hyst_step.copy())
        # sort the list based on the amplitude
        hx_amps = np.zeros(len(hyst_pole))
        for i, df in enumerate(hyst_pole):
            hx = df['hx'+poles_dict[pole]]
            hx_amps[i] = hx.max() - hx.min()
        # sort the hyst pole based on amps
        hyst_pole = [x for _,x in sorted(zip(hx_amps,hyst_pole))]

        # add to the pole list
        hyst_all.append(hyst_pole)

In [15]:
# plot the bounding loop
bounding_loop = hyst_all[pole][-1]
x_full = bounding_loop['hp'+poles_dict[pole]].values
y_full = bounding_loop['hx'+poles_dict[pole]].values

poly_coeff = np.polyfit(x_full, y_full, deg=7)
polyfit = np.poly1d(poly_coeff)
polyder_coeff = np.polyder(poly_coeff, 1)
polyder = np.poly1d(polyder_coeff)

plt.close('all')
plt.plot(x_full, y_full, label='data')
plt.plot(x_full, polyfit(x_full), label='fit')
plt.plot(x_full, polyder(x_full), label='derivative')
plt.legend()
# plt.plot(x_full / np.max(x_full), label='hallprobe')
# plt.plot(y_full / np.max(y_full), label = 'hexapole')

In [16]:
# plot everything
# plt.close()
plt.figure(figsize=(16, 8))
for pole, hyst_pole in enumerate(hyst_all):
    plt.subplot(1, len(poles), pole+1)
    for i, hyst_step in enumerate(hyst_pole):
        plt.plot(hyst_step['hp'+poles_dict[pole]],hyst_step['hx'+poles_dict[pole]], label='step{}'.format(i))
        plt.xlabel('Measured field [V]')
        plt.ylabel('Applied current [A]')
#     plt.legend()
    plt.title('Pole {}'.format(poles_dict[pole]))

plt.tight_layout()
# plt.savefig(os.path.join(out_folder, 'Magnet minor loops flipped.png'))

In [18]:
with open('hyst_poly_coeff.p', 'wb') as f:
    pickle.dump({'loop':poly_coeff, 'deriv': polyder_coeff}, f)

### Fitting the bounding loop (f<sub>a</sub>)

In [88]:
plt.close('all')
plt.figure(figsize=(18, 7))
main_branch_fits = []

for pole in poles:
    print('\nPole '+poles_dict[pole])
    bounding_loop = hyst_all[pole][-1]
    x_full = bounding_loop['hp'+poles_dict[pole]].values
    y_full = bounding_loop['hx'+poles_dict[pole]].values
    # take only the top half
    x = x_full[:int(np.floor(len(x_full)/2))]
    y = y_full[:int(np.floor(len(y_full)/2))]

    # fit the hysteresis
    idx = np.argsort(x)
    x, y = x[idx], y[idx]
    spl = interpolate.UnivariateSpline(x, y, s=0.2)
#     spl = interpolate.InterpolatedUnivariateSpline(x, y)
    main_branch_fits.append(spl)
    y_fit = spl.__call__(x)
    
    # do the error analysis
    err = np.abs((y_fit - y))
    idx = np.argmax(err)
    maxerr = err[idx]
    maxrelerr = np.abs(np.max(np.abs(err/y)))
    stderr = np.sqrt(np.mean(err**2))
    s = 'Maximum error: {:.3f}V\nStd error: {:.3f}V'.format(maxerr, stderr)
    print(s)
#     print(y[idx])
    
#     coeff = np.polyfit(x, y, deg=12)
#     f_a = np.poly1d(coeff)
    
    plt.subplot(1, len(poles), pole+1)
    plt.plot(x_full, y_full, label='Measurement')
    x_tst = np.linspace(-1.9, 1.9, 1000)
    plt.plot(x_tst, spl.__call__(x_tst), label='Fit')
    plt.legend()
    plt.ylabel('Applied current [A]')
    plt.xlabel('Measured field [V]')
    plt.title('Pole '+poles_dict[pole])
    plt.text(-2, 6, s, )
    
plt.tight_layout()
# plt.savefig(os.path.join(out_folder, 'Fitting bounding loop all poles.png'))


Pole A
Maximum error: 0.060V
Std error: 0.022V

Pole B
Maximum error: 0.056V
Std error: 0.022V

Pole C
Maximum error: 0.064V
Std error: 0.022V


### Fitting the minor loops f<sub>ab

In [89]:
# for each of the transition curves, get the value maximum value it gets to
transition_curves = []
for pole in poles:
    # maximum value of the minor loops
    minor_loops = hyst_all[pole]
    # a = np.array([minor_loop['hx'+poles_dict[pole]].max() for minor_loop in minor_loops])
    # get the list of returning loops
    tran_curves = []
    for ml in minor_loops:
        y = ml['hx'+poles_dict[pole]]
        x = ml['hp'+poles_dict[pole]]
        # get the maximum value of the loop
        a = x.max()
        # take only the bottom half
        b = x[int(np.floor(len(x)/2)):].values
        f_ab = y[int(np.floor(len(y)/2)):].values
        tran_curves.append(np.stack((a*np.ones(b.size), b, f_ab), axis=1))
    transition_curves.append(np.concatenate(tran_curves, axis=0))

In [90]:
pole = 0
tran_pole = transition_curves[pole]
x, y, z = tran_pole[:, 0], tran_pole[:, 1], tran_pole[:, 2]
# try to optimize the metaparameters
bestparams = [1, 1, 1]
minerr = 1000
for s in range(3, 40):
    for kx in range(1, 5):
        for ky in range(1, 5):
            fit = interpolate.SmoothBivariateSpline(x, y, z, s=s, kx=kx, ky=ky)
            z_eval = fit.ev(x, y)
            # do the error analysis
            err = np.abs((z - z_eval))
            stderr = np.sqrt(np.mean(err**2))
            if stderr < minerr:
                bestparams = [s, kx, ky]
                minerr = stderr
                print(minerr)
                print(bestparams)
print('Done!')


0.028748273728068825
[6, 4, 4]


c:\users\lukas\appdata\local\programs\python\python38-32\lib\site-packages\scipy\interpolate\fitpack2.py:1065: UserWarning: ier=1499
  warnings.warn(message)
c:\users\lukas\appdata\local\programs\python\python38-32\lib\site-packages\scipy\interpolate\fitpack2.py:1065: UserWarning: ier=988
  warnings.warn(message)


Done!


In [92]:
plt.close('all')
fig = plt.figure(figsize=(18, 9))
transition_fits = []
poles = [0, 1, 2]
for pole in poles:
    print('\nPole '+poles_dict[pole])
    tran_pole = transition_curves[pole]
    x, y, z = tran_pole[:, 0], tran_pole[:, 1], tran_pole[:, 2]
    grid_x, grid_y = np.mgrid[-2:2:200j, -2:2:200j]
#     fit = interpolate.interp2d(x, y, z, kind='cubic')
                
    fit = interpolate.SmoothBivariateSpline(x, y, z, s=bestparams[0], kx=bestparams[1], ky=bestparams[2])
    transition_fits.append(fit)
    grid_z = fit.__call__(grid_x[:, 0], grid_y[0, :])
#     grid_z = np.reshape(fit(grid_x.flatten(), grid_y.flatten()), grid_x.shape)

    # for illustration purposes, make the grid z be 0 outside of the relevant area
    idx = grid_x<grid_y
    grid_z[idx] = 0
    
    grid_z = np.reshape(grid_z, grid_x.shape)

    ax = fig.add_subplot(1, len(poles), 1+pole, projection='3d')
    ax.scatter(x, y, z)
    ax.plot_surface(grid_x, grid_y, grid_z, alpha=0.5, color='red')
    ax.set_zlim([-10, 10])
    ax.set_xlabel(r'$\alpha$ [V]')
    ax.set_ylabel(r'$\beta$ [V]')
    ax.set_zlabel(r'$f_{\alpha\beta}$ [V]')
    

    ax.view_init(15, -140)
    z_eval = fit.ev(x, y)
    
    
    # do the error analysis
    err = np.abs((z - z_eval))
    idx = np.argmax(err)
    maxerr = err[idx]
    maxrelerr = np.abs(np.max(np.abs(err/z)))
    stderr = np.sqrt(np.mean(err**2))
    s = 'Maximum error: {:.2f}V\nStd error: {:.3f}V'.format(maxerr, stderr)
    print(s)
#     print('Value at max rel: {}'.format(z[np.argmax(np.abs(err/z))]))
    plt.title('Pole {}\n'.format(poles_dict[pole])+s)

plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Fitting transition curves all poles.png'))


Pole A
Maximum error: 0.11V
Std error: 0.029V

Pole B
Maximum error: 0.08V
Std error: 0.029V

Pole C
Maximum error: 0.08V
Std error: 0.029V


### Define the triangle function F(a, b)

In [8]:
# define the triangle functions:
F_functions = [lambda a, b, f_a=f_a, f_ab=f_ab: (f_a.__call__(a) - f_ab.ev(a, b))/2 for f_a, f_ab in zip(main_branch_fits, transition_fits)]
# check that it gives sensible results
print(F_functions[0](1.8, 1.8))
print(main_branch_fits[1].__call__(1.8))
print(transition_fits[1].ev(1.8, -1.8))

0.009823854425893508
9.324649904682046
-9.169066344840166


### Create functions for updating the line points of the Prezbiach model

In [3]:
from numba import jit

@jit(nopython=True)
def update_line(L, x):
    # first find out if we went up or down
    if L[-1, 0] < x:
        # we went up, get rid of the history of all elements that have alpha smaller than x
        L[L[:, 0] < x, 0] = x
        # add the newest element
        L = np.vstack((L, np.array([[x, x]])))
        # every other element needs to have a different alpha, otherwise need to get rid of it
        keep = np.zeros(L.shape[0]).astype(np.bool_)
        keep[0] = True
        keep[-1] = True
        for i in range(L.shape[0]-2):
            if L[i, 0] != L[i+1, 0]:
                keep[i] = True
                keep[i+1] = True
        L = L[keep, :]
        return L
    if L[-1, 0] > x:
        # we went up, get rid of the history of all elements that have alpha smaller than x
        L[L[:, 1] > x, 1] = x
        # add the newest element
        L = np.vstack((L, np.array([[x, x]])))
        # every other element needs to have a different alpha, otherwise need to get rid of it
        keep = np.zeros(L.shape[0]).astype(np.bool_)
        keep[0] = True
        keep[-1] = True
        for i in range(L.shape[0]-2):
            if L[i, 1] != L[i+1, 1]:
                keep[i] = True
                keep[i+1] = True
        L = L[keep, :]
        return L
    return L    

def get_signal_value(L, F_fun):
    """Gets the value of the signal having the points of the Prezbiach line and the integral functions"""
    # get alphas and betas
    alpha, beta = L[:, 0], L[:, 1]
    # get all the horisontal lines
    indx_hor = (beta[1:] - beta[:-1]) != 0
#     lenh = np.sum(indx_hor)
#     print(lenh)
    # calcluate the signal
    signal = -F_fun(alpha[0], beta[0])
    F_kk = F_fun(alpha[:-1][indx_hor], beta[:-1][indx_hor])
    F_kk1 = F_fun(alpha[1:][indx_hor], beta[1:][indx_hor])
    signal += 2*np.sum(F_kk - F_kk1)    
    return signal

def reconstruct_signal(signal, L_start, F_fun):
    # define the starting line
    L = L_start
    signal_out = np.zeros(len(signal))
    for i, x in enumerate(signal):
        L = update_line(L, x)
        signal_out[i] = get_signal_value(L, F_fun)
    return signal_out


### Generating the degauss L

In [187]:
degauss_file = r'C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\degauss_signal.h5'

with h5py.File(degauss_file, 'r') as f:
    degauss_signal =  pd.DataFrame(f['degaussing/hallprobe/data'][:])
    bins = np.arange(degauss_signal[0].iloc[0], degauss_signal[0].iloc[-1], 1/100)
    degauss_signal['bins'] = pd.cut(degauss_signal[0], bins, right=False)
    degauss_signal = degauss_signal.groupby('bins').mean().reset_index().drop(labels=[0,'bins'], axis=1)
    degauss_signal.columns = ['hpA', 'hpB', 'hpC']

degauss_l = []
start_l1 = []
# start_l = [np.array([[hp_minmax[pole][1], hp_minmax[pole][0]], [hp_minmax[pole][0], hp_minmax[pole][0]]]) 
#           for pole in poles]
start_l = start_l_best.copy()
for pole in poles:
#     x = hyst_all[pole][-1]['hp'+poles_dict[pole]].values
#     y = hyst_all[pole][-1]['hx'+poles_dict[pole]].values
#     mx, mn = np.max(x), np.min(x)
#     L = (np.array([[mx, mn], [mn, mn]])).astype(np.float)
#     start_l1.append(L)
    dg_onepole = degauss_signal['hp{}'.format(poles_dict[pole])].values

    L = start_l[pole].copy()
    for x in dg_onepole:
        L = update_line(L, x)
    degauss_l.append(L.copy())

# use the error in the degaussing as a displacement in the fit. 
#I believe that this is needed because of the wrong starting point choice
fitting_displacements = []
for pole in poles:
     fitting_displacements.append(-get_signal_value(degauss_l[pole], F_functions[pole]))
print(fitting_displacements)

### Try to reconstruct the signal

In [184]:
# optimise the start_l
start_l = [np.array([[hp_minmax[pole][1], hp_minmax[pole][0]], [hp_minmax[pole][0], hp_minmax[pole][0]]]) 
          for pole in poles]
start_l_best = start_l.copy()
for pole in poles:
    x = hyst_all[pole][-1]['hp'+poles_dict[pole]].values
    y = hyst_all[pole][-1]['hx'+poles_dict[pole]].values
#     stderr_min = 1000
    dg_signal_min = 1000
    for mx in np.linspace(-0.2, 0, 40):
        for mn in np.linspace(-0.2, 0, 40):
            start_l_pole =  start_l[pole].copy()
            start_l_pole[0, 0] += mx
            start_l_pole[0, 1] += mn
            start_l_pole[-1, :] += mn
#             y_pred = reconstruct_signal(x,start_l_pole.copy(), F_functions[pole])
#             err = y - y_pred
#             stderr = np.sqrt(np.mean(err**2))
            degauss_l[pole][0, :] = start_l_pole[0, :]
            degauss_l[pole][1, 1] = start_l_pole[1, 1]
            dg_signal = np.abs(get_signal_value(degauss_l[pole], F_functions[pole]))
            if dg_signal < dg_signal_min:
                dg_signal_min = dg_signal
                start_l_best[pole] = start_l_pole
    print(dg_signal_min)

0.005567347182173776
0.008369808846827809
0.08462369830784056


In [ ]:
degauss_l

In [186]:
start_l_best

[array([[ 1.82595961, -2.07175638],
        [-2.07175638, -2.07175638]]), array([[ 1.78248633, -2.03289505],
        [-2.03289505, -2.03289505]]), array([[ 1.76565731, -2.01216834],
        [-2.01216834, -2.01216834]])]

In [189]:
import time
pole = 0
# fitting_displacements = []

plt.close('all')
fig = plt.figure(figsize=(18, 7))
errcolor = 'tab:green'

# start_l = [np.array([[hp_minmax[pole][1] - 0.02, hp_minmax[pole][0]], [hp_minmax[pole][0], hp_minmax[pole][0]]]) 
#           for pole in poles]
for pole in poles:
    x = hyst_all[pole][-1]['hp'+poles_dict[pole]].values
    y = hyst_all[pole][-1]['hx'+poles_dict[pole]].values
    y_pred = reconstruct_signal(x, start_l_best[pole].copy(), F_functions[pole]) + fitting_displacements[pole]

    displacement = np.mean(y-y_pred)
    print(displacement)

    idx = np.ones(x.size).astype(bool)
    
    ax = fig.add_subplot(1, len(poles), 1+pole)
    ax.plot(x[idx], y[idx], label='Data')
    ax.plot(x[idx], y_pred[idx], label='Fit')
    ax.set_ylabel('Applied signal [V]')
    ax.set_xlabel('Measured field [V]')
    
    # add the error plot on the other axes
    ax2 = ax.twinx()
    ax2.plot(x, y_pred - y, color=errcolor, alpha=0.8,  label='Error')
    ax2.set_ylabel('Error [V]', color=errcolor)
    ax2.set_ylim([-0.5, 0.5])
    ax2.tick_params(axis='y', labelcolor=errcolor)
    
    # do the error analysis
    err = np.abs((y_pred - y))
    idx = np.argmax(err)
    maxerr = err[idx]
    maxrelerr = np.abs(np.max(np.abs(err/y)))
    stderr = np.sqrt(np.mean(err**2))
    s = 'Maximum error: {:.2f}V\nStd error: {:.3f}V'.format(maxerr, stderr)
    print(s)
    plt.title('Pole {}\n'.format(poles_dict[pole])+s)
    
    ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Fitting hysteresis_optimizedstartl.png'.format(pole)))

-0.04056651052540228
Maximum error: 0.16V
Std error: 0.078V
-0.05167865732622829
Maximum error: 0.13V
Std error: 0.066V
0.04958923824635043
Maximum error: 0.12V
Std error: 0.059V


In [180]:
start_l

[array([[ 1.83621602, -1.87175638],
        [-1.87175638, -1.87175638]]), array([[ 1.80299915, -1.83289505],
        [-1.83289505, -1.83289505]]), array([[ 1.76565731, -1.81216834],
        [-1.81216834, -1.81216834]])]

### Compare to fitting without hysteresis

In [35]:
# import the hysteresis fits
with open('fields2current_coeff.p', 'rb') as f:
    h2i_coeff = pickle.load(f)
# define functions for going from fields to required current
i2h_functions = [np.poly1d(c) for c in h2i_coeff]
y_prednohyst = -i2h_functions[pole](x)
plt.close('all')
fig = plt.figure(figsize=(10, 10))
plt.plot(x, y, label='Data')
plt.plot(x, y_pred, label='Fit hysteresis')
plt.plot(x, y_prednohyst, label='Fit no hysteresis')
plt.plot(x, y-y_pred, label='Hysteresis fit error')
plt.plot(x, y-y_prednohyst, label='No hysteresis fit error')
plt.xlabel('Applied signal [V]')
plt.ylabel('Measured field [V]')
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(out_folder, 'Fitting hysteresis.png'))

## Save all the results

In [190]:
# check if everything makes sense
for pole in poles:
    print(get_signal_value(degauss_l[pole], F_functions[pole])+fitting_displacements[pole])

0.0
0.0
0.0


In [191]:
start_l = start_l_best

hysteresis_fits = {'main_branch':main_branch_fits, 
                   'transition_branches':transition_fits,
                  'start_l':start_l,
                  'fitting_displacements':fitting_displacements,
                  'degauss_l':degauss_l}
with open('hysteresis_fits.p', 'wb') as f:
    pickle.dump(hysteresis_fits, f)
print('Saved')

Saved


### Combine with RL parameters to have only one pickle file

In [193]:

with open('rl_coeff.p', 'rb') as f:
    rl_coeff = pickle.load(f)
print(rl_coeff)
magnet_calib = hysteresis_fits.copy()
magnet_calib['R'] = [rl_coeff[pole]['R'] for pole in poles]
magnet_calib['L'] = [rl_coeff[pole]['L'] for pole in poles]
with open('magnet_calib.p', 'wb') as f:
    pickle.dump(magnet_calib, f)
print('saved')

[{'R': 2.118457140120971, 'L': 0.20069968460466536}, {'R': 2.108782231622595, 'L': 0.20365503550736846}, {'R': 2.07589810652447, 'L': 0.19539368897169337}]
saved


In [3]:
with open('magnet_calib.p', 'rb') as f:
    hysteresis_fits = pickle.load(f)
main_branch_fits = hysteresis_fits['main_branch']
transition_fits = hysteresis_fits['transition_branches']
start_l = hysteresis_fits['start_l']
fitting_displacements = hysteresis_fits['fitting_displacements']
degauss_l = hysteresis_fits['degauss_l']

In [11]:
from numba import njit, jit
@jit(forceobj=True)
def jitted_mb(x):
    return main_branch_fits[0](x)

In [13]:
jitted_mb(1)

array(2.22222033)

In [15]:
# define the triangle functions
F_functions = [lambda a, b, f_a=f_a, f_ab=f_ab: (f_a.__call__(a) - f_ab.ev(a, b))/2 for f_a, f_ab in zip(main_branch_fits, transition_fits)]

@jit(forceobj=True, fastmath=True)
def jit_F_functions(a, b, pole):
    return F_functions[pole](a, b)
# check that it gives sensible results
print(F_functions[0](1.8, 1.8))
print(jit_F_functions(1.8, 1.8, 0))
print(main_branch_fits[1].__call__(1.8))
print(transition_fits[1].ev(1.8, -1.8))

0.009823854425893508
0.009823854425893508
9.324649904682046
-9.169066344840166


In [24]:
import time
t0 = time.time()
for i in range(50000):
    F_functions[0](1.2, 0)
print(time.time() - t0)
t0 = time.time()
for i in range(50000):
    jit_F_functions(1.2, 0, 0)
print(time.time() - t0)

0.8681409358978271
0.9709093570709229


In [17]:
pole = 0
F_functions[pole](3, -3)

121.92493344419641

In [19]:
with h5py.File('test.h5', 'w') as f:
    f.create_dataset("name", (100,), dtype=h5py.string_dtype())